In [20]:
# import h5py
import numpy as np
import os, datetime, glob, cv2
from nexusformat.nexus import *
from pathlib import Path
import re, locale, configparser
locale.setlocale(locale.LC_ALL, '')  #übernehmen der Systemeinstellungen.

shotdir = os.path.abspath("") + r'\waaterdroplett experiment\Shots\001' # for now just one shot
subdirs = [ item for item in os.listdir(shotdir) if os.path.isdir(os.path.join(shotdir, item)) ]

In [21]:
# NI Vision has some special behavior if saving PNGs.... 
# See https://stackoverflow.com/questions/77454243/imageio-imread-retains-the-original-grayscale-values-when-reading-a-png-file/
# and https://forums.ni.com/t5/Machine-Vision/How-do-you-load-signed-16-bit-png-file-outside-of-labVIEW/td-p/4247192

def read_NI_PNG(file_path):
    png = Path(file_path).read_bytes()
    sBIToffset = png.find(b'sBIT')
    scAl = png.find(b'scAl') # scAl is a constant offset
    values=[0]

    if sBIToffset>0:
        # 4 bytes before "sBIT" tell us how many values there are - could be 1-4 values
        nValues = int.from_bytes(png[sBIToffset-4:sBIToffset], byteorder="big")
        values  = list(png[sBIToffset+4:sBIToffset+4+nValues])

    offset=0
    if scAl>0:
        offset = 65536 - int.from_bytes(png[scAl+4:scAl+6], "big")

    image=cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    if len(values)==1:
        shift = 16-values[0]
        if shift<16:
            image=(image>> shift).astype(np.int16)
        if offset>0:
            image=image-offset
    return image
  

In [22]:
#Reads the .data file and maps to a Python dict.
def map_imagereport_to_dict(imagereportfile):
    config = configparser.ConfigParser(delimiters="=")
    config.read(imagereportfile)
    dictionary = {}
    for section in config.sections():
        dictionary[section] = {}
        for option in config.options(section):
            dictionary[section][option] = config.get(section, option)
    return (dictionary)

In [23]:
def add_image_to_entry(nx, entry, image, md={}):
    nx[entry + '/instrument'] = NXinstrument(NXdetector())
    nx[entry + '/instrument/detector/image'] = NXfield(image, units='counts',
                                                    compression='gzip')
    nx[entry +'/data'] = NXdata()
    nx[entry +'/data'].makelink(nx[entry +'/instrument/detector/image'])
    nx[entry +'/data'].nxsignal = nx[entry + '/data/image']

    if len(md) > 0:
        metadata = nx[entry + '/instrument/metadata'] = NXcollection()
        for k, v in md.items():
            if len(v) > 0:
                for k1, v1 in v.items():
                     metadata[(k+":"+k1).replace('/',':')] = v1
            else:
                metadata[k.replace('/',':')] = v

In [24]:
def add_cam_params(nxfile, subentry, camsettings):
    camAttrOfInterest=['exposuretimeabs', 'exposuremode', 'gain', 'gainraw','gainauto']  # `caio´
    mapCamAttr2Unit={'exposuretimeabs': 'us'}
    camPPath = subentry+'/camera_parameters'
    nxfile[camPPath]=NXparameters()
    for k, v in camsettings.items():
        for caoi in camAttrOfInterest:
            result=re.search(caoi+'$', k)  #Finde Strings die mit CAIO enden (regex: $).
            if result: 
                try:
                    pVal = locale.atof(v)
                    if caoi in mapCamAttr2Unit:
                        unit = mapCamAttr2Unit[caoi]    
                        nxfile[camPPath+'/'+caoi]=NXfield(pVal, units=unit)
                    else: 
                        nxfile[camPPath+'/'+caoi]=NXfield(pVal)
                except ValueError:
                    pass
    

In [25]:
def add_image_preprocessing_params(nxfile, subentry, settings):
    imgPPath=subentry+'/image_preprocessing'
    nxfile[imgPPath]=NXparameters()
    
    v = locale.atof(settings['stretch x']) 
    nxfile[imgPPath+'/stretch_x']=NXfield(v)

    v = (settings['flip v']=='TRUE')
    nxfile[imgPPath+'/flip_v']=NXfield(v)
    
    v = (settings['flip h']=='TRUE')
    nxfile[imgPPath+'/flip_h']=NXfield(v)

In [26]:
def add_measurement_params(nxfile, subentry, params):
    pass

In [27]:
# Create a NXentry und NXsubentries for all cams. For now just for one shot.
entry_name="shot_001"
fileName = "ShotContainer.hdf5"
with nxopen(fileName,"w") as f:
    f[entry_name]=NXentry()
    for dir in subdirs:
        subentry = entry_name + '/'+ dir 
        f[subentry]=NXsubentry()
        try:
            reportfile = glob.glob(shotdir+'/'+dir +'/*.data')[0]
            metadata = map_imagereport_to_dict(reportfile)
            try:
                camsettings = metadata['CameraSettings']
                add_cam_params(f,subentry,camsettings)
            except KeyError:
                pass
        except IndexError:
            reportfile = ""
        try: 
            imgPP=metadata['Image Preprocessing ']
            add_image_preprocessing_params(f, subentry, imgPP)
        except KeyError:
            pass
        try:
            imgfile = glob.glob(shotdir+'/'+dir +'/*.png')[0]    
            img = read_NI_PNG(imgfile)
            #add_image_to_entry(f,subentry,img, metadata)
            add_image_to_entry(f,subentry,img)
        except IndexError:
            metadata={}
    print(f.tree)

c:\Users\AlexK\work\HELPMI\NeXus-for-HELPMI\POLARIS_ExpData_Example\waaterdroplett experiment\Shots\001
c:\Users\AlexK\work\HELPMI\NeXus-for-HELPMI\POLARIS_ExpData_Example\waaterdroplett experiment\Shots\001


c:\Users\AlexK\work\HELPMI\NeXus-for-HELPMI\POLARIS_ExpData_Example\waaterdroplett experiment\Shots\001
c:\Users\AlexK\work\HELPMI\NeXus-for-HELPMI\POLARIS_ExpData_Example\waaterdroplett experiment\Shots\001
c:\Users\AlexK\work\HELPMI\NeXus-for-HELPMI\POLARIS_ExpData_Example\waaterdroplett experiment\Shots\001
c:\Users\AlexK\work\HELPMI\NeXus-for-HELPMI\POLARIS_ExpData_Example\waaterdroplett experiment\Shots\001
c:\Users\AlexK\work\HELPMI\NeXus-for-HELPMI\POLARIS_ExpData_Example\waaterdroplett experiment\Shots\001
root:NXroot
  shot_001:NXentry
    Farfield:NXsubentry
      camera_parameters:NXparameters
        exposuremode = 1.0
        exposuretimeabs = 300.0
          @units = 'us'
        gain = 0.0
        gainauto = 1.0
      data:NXdata
        @signal = 'image'
        image -> /shot_001/Farfield/instrument/detector/image
      image_preprocessing:NXparameters
        flip_h = False
        flip_v = False
        stretch_x = 1.0
      instrument:NXinstrument
        detector:N